In [1]:
import requests
import json
import pandas as pd
from threading import Thread
import subprocess
import os
import time
from datetime import datetime

In [2]:
def get_directory_server():
    """Retorna o diretorio completo do arquivo web-server"""
    dirc = r'\ada_project-main\web-server.js'
    return str(os.getcwd())+dirc

In [3]:
def run_server(dirc):
  """Inicia o servidor como um subprocesso"""
  print('Server Started \n')
  output = subprocess.run(["node", dirc])
  print('Server Closed')
  
def kill_server():
  os.system('cmd /c taskkill /f /im node.exe')

In [61]:
#Inicia o servidor através dos scripts em uma thread, ignorar esta célula caso dê erro e iniciar o servidor manualmente
server = Thread(target=run_server, args=(get_directory_server(),)) #you can create as many threads as you need
server.start()
def fill_dataset(URL: str='http://localhost:3000/api/ep1'):
    """Faz as requisições na API até a data atual e transforma os dados recebidos para um dataset"""
    df=pd.DataFrame(columns=['id', 'date', 'n_week', 'week_year', 'month_year']+[f'prod_{n}' for n in range(16)])
    aux=1
    while True:
        request = pd.DataFrame(requests.get('http://localhost:3000/api/ep1').json())
        if (len(request)>0 and request['date'][0]>datetime.timestamp(datetime.now())):
            break
        request['n_week']= aux
        df=pd.concat([df, request])
        aux+=1

    df['date'] = pd.to_datetime(df['date'], unit='s')
    df['month_year'] = df['date'].dt.strftime('%m-%Y')
    df['week_year'] = df['date'].dt.strftime('%U-%Y')
    df = df.fillna(0)
    return df
#Cria o dataset com todas as requisições e salva em um csv
df= fill_dataset()
df.to_csv(r'Dados\all_sales.csv')
kill_server()

Server Started 

Server Closed
